In [53]:
!pip install spacy

You should consider upgrading via the '/Users/yulinhu/miniconda3/bin/python -m pip install --upgrade pip' command.


In [62]:
import sys
import nltk
from nltk.stem import PorterStemmer
import spacy
import numpy as np
from queue import PriorityQueue
from numpy.linalg import norm
import math


training_file = '%_nombank.clean.train'
test_file = '%_nombank.clean.test'
file = open(training_file,"r")
lines = file.readlines()
file.close()

ps = PorterStemmer()
nlp = spacy.load("en_core_web_md")

f = open("training.feature", "w")
training_features = []
for line in lines:
    t = line.split()
    training_features.append(t)

sentence = []
arg1_index = -1
# global_arg1_vector = np.zeros(300,)
global_unigram_embed = np.zeros(300,)
global_forward_bigram = np.zeros(300,)
global_backward_bigram = np.zeros(300,)
global_forward_trigram = np.zeros(300,)
global_backward_trigram = np.zeros(300,)
count = 0
for i in range(len(training_features)):
    t = training_features[i]
    if len(t) == 0:
        text = ' '.join(sentence)
#         print(text, arg1_index)
        doc = nlp(text)
        # length = len(sentence)

        # global arg1 vector
        # arg1_vector = doc[arg1_index].vector
        # global_arg1_vector += arg1_vector

        # global n-gram vector
        # Vector for ARG1 unigrams
        unigram_embed = doc[arg1_index].vector
        global_unigram_embed += unigram_embed
        # Vector for forward ARG1 bigrams
        forward_bigram = doc[arg1_index:arg1_index+2].vector
        global_forward_bigram += forward_bigram
        # Vector for backward ARG1 bigrams
        backward_bigram = doc[arg1_index-1:arg1_index+1].vector
        global_backward_bigram += backward_bigram
        # Vector for forward ARG1 trigrams
        forward_trigram = doc[arg1_index:arg1_index+3].vector
        global_forward_trigram += forward_trigram
        # Vector for backward ARG1 trigrams
        backward_trigram = doc[arg1_index-2:arg1_index+1].vector
        global_backward_trigram += backward_trigram


        # line = "\n"
        # f.write(line)
        sentence.clear()
        count += 1
        continue

    token = t[0]
    POS_tag = t[1]
    BIO_tag = t[2]
    role_label = ""
    class_of_predicate = ""
    stemmed_token = ps.stem(token)
    if len(t) > 5:
        role_label = t[5]
        if role_label == "PRED":
            class_of_predicate = t[6]

    if len(t) > 5:
            role_label = t[5]
            if role_label == "ARG1":
                arg1_index = int(t[3])

    sentence.append(token)

# global_arg1_vector /= count
global_unigram_embed /= count
global_forward_bigram /= count
global_backward_bigram /= count
global_forward_trigram /= count
global_backward_trigram /= count
# print(global_unigram_embed)
# print(global_forward_bigram)
# print(global_backward_bigram)
# print(global_forward_trigram)
# print(global_backward_trigram)

sentence = []
text_lines = []
arg1_index = -1
for i in range(len(training_features)):
    t = training_features[i]
    if len(t) == 0:
        # Embeddings of N-grams
        text = ' '.join(sentence)
        # print(text)
        doc = nlp(text)
        length = len(sentence)
        pq_unigram = PriorityQueue()
        pq_forward_bigram = PriorityQueue()
        pq_backward_bigram = PriorityQueue()
        pq_forward_trigram = PriorityQueue()
        pq_backward_trigram = PriorityQueue()
        for j in range(length):
            # print(doc[j])
            # Vector for ARG1 unigrams
            unigram_embed = doc[j].vector
            # Vector for forward ARG1 bigrams
            forward_bigram = doc[j:j+2].vector
            # Vector for backward ARG1 bigrams
            backward_bigram = doc[j-1:j+1].vector
            # Vector for forward ARG1 trigrams
            forward_trigram = doc[j:j+3].vector
            # Vector for backward ARG1 trigrams
            backward_trigram = doc[j-2:j+1].vector

            # forward_bigram_similarity = 0
            # backward_bigram_similarity = 0
            # forward_trigram_similarity = 0
            # backward_trigram_similarity = 0

            unigram_embed_similarity = np.dot(global_unigram_embed, unigram_embed)/(norm(global_unigram_embed)*norm(unigram_embed))
            # if not (forward_bigram == np.zeros(300,)).all():
            forward_bigram_similarity = np.dot(global_forward_bigram, forward_bigram)/(norm(global_forward_bigram)*norm(forward_bigram))
            # if not (backward_bigram == np.zeros(300,)).all():
            backward_bigram_similarity = np.dot(global_backward_bigram, backward_bigram)/(norm(global_backward_bigram)*norm(backward_bigram))
            # if not (forward_trigram == np.zeros(300,)).all():
            forward_trigram_similarity = np.dot(global_forward_trigram, forward_trigram)/(norm(global_forward_trigram)*norm(forward_trigram))
            # if not (backward_trigram == np.zeros(300,)).all():
            backward_trigram_similarity = np.dot(global_backward_trigram, backward_trigram)/(norm(global_backward_trigram)*norm(backward_trigram))

            if not math.isnan(unigram_embed_similarity):
                pq_unigram.put(-unigram_embed_similarity)
            if not math.isnan(forward_bigram_similarity):
                pq_forward_bigram.put(-forward_bigram_similarity)
            if not math.isnan(backward_bigram_similarity):
                pq_backward_bigram.put(-backward_bigram_similarity)
            if not math.isnan(forward_trigram_similarity):
                pq_forward_trigram.put(-forward_trigram_similarity)
            if not math.isnan(backward_trigram_similarity):
                pq_backward_trigram.put(-backward_trigram_similarity)


            text_line = text_lines[j]
            text_line += "unigram_embed_similarity=" + str(unigram_embed_similarity) + "\tforward_bigram_similarity=" + str(forward_bigram_similarity) + \
                            "\tbackward_bigram_similarity=" + str(backward_bigram_similarity) + "\tforward_trigram_similarity=" \
                        + str(forward_trigram_similarity) + "\tbackward_trigram_similarity=" + str(backward_trigram_similarity) + "\t"
            if arg1_index == j:
                text_line += "True\n"
            else:
                text_line += "False\n"
            
            # print(text_line)
            f.write(text_line)
        
        # print("ranking")
        top_unigram = ""
        top_forward_bigram = ""
        top_backward_bigram = ""
        top_forward_trigram = ""
        top_backward_trigram = ""
        # print(pq_unigram.qsize())
        # print(pq_forward_bigram.qsize())
        # print(pq_backward_bigram.qsize())
        # print(pq_forward_trigram.qsize())
        # print(pq_backward_trigram.qsize())

        for n in range(5):
            if not pq_unigram.empty():
                top_unigram += "top_" + str(n+1) + "_embed_unigram="+str((-1)*pq_unigram.get()) + "\t"
            if not pq_forward_bigram.empty():
                top_forward_bigram += "top_" + str(n+1) + "_embed_forward_bigram=" + str((-1)*pq_forward_bigram.get()) + "\t"
            if not pq_backward_bigram.empty():
                top_backward_bigram += "top_" + str(n+1) + "_embed_backward_bigram=" + str((-1)*pq_backward_bigram.get()) + "\t"
            if not pq_forward_trigram.empty():
                top_forward_trigram += "top_" + str(n+1) + "_embed_forward_trigram=" + str((-1)*pq_forward_trigram.get()) + "\t"
            if not pq_backward_trigram.empty():
                top_backward_trigram += "top_" + str(n+1) + "_embed_backward_trigram=" + str((-1)*pq_backward_trigram.get()) + "\t"

        top_unigram += "\n"
        top_forward_bigram += "\n"
        top_backward_bigram += "\n"
        top_forward_trigram += "\n"
        top_backward_trigram += "\n"
        f.write(top_unigram)
        f.write(top_forward_bigram)
        f.write(top_backward_bigram)
        f.write(top_forward_trigram)
        f.write(top_backward_trigram)




        line = "\n"
        f.write(line)
        sentence.clear()
        text_lines.clear()
        # print("end")
        continue
    token = t[0]
    POS_tag = t[1]
    BIO_tag = t[2]
    role_label = ""
    class_of_predicate = ""
    stemmed_token = ps.stem(token)
    if len(t) > 5:
        role_label = t[5]
        if role_label == "PRED":
            class_of_predicate = t[6]
    prev_info1 = ""
    prev_info2 = ""
    if training_features[i-1]:
        prev_t1 = training_features[i-1]
        prev_token1 = prev_t1[0]
        prev_POS_tag1 = prev_t1[1]
        prev_BIO_tag1 = prev_t1[2]
        stemmed_prev_token1 = ps.stem(prev_token1)
        prev_role_label1 = ""
        prev_class_of_predicate1 = ""
        if len(prev_t1) > 5:
            prev_role_label1 = prev_t1[5]
            if prev_role_label1 == "PRED":
                prev_class_of_predicate1 = prev_t1[6]

        prev_info1 = "word_back_1=" + prev_token1 + "\tPOS_back_1=" + prev_POS_tag1 + "\tBIO_back_1=" + prev_BIO_tag1 + "\tstemmed_word_back_1=" \
                        + stemmed_prev_token1 + "\t" 

        if training_features[i-2]:
            prev_t2 = training_features[i-2]
            prev_token2 = prev_t2[0]
            prev_POS_tag2 = prev_t2[1]
            prev_BIO_tag2 = prev_t2[2]
            stemmed_prev_token2 = ps.stem(prev_token2)
            prev_role_label2 = ""
            prev_class_of_predicate2 = ""
            if len(prev_t2) > 5:
                prev_role_label2 = prev_t2[5]
                if prev_role_label2 == "PRED":
                    prev_class_of_predicate2 = prev_t2[6]

            prev_info2 = "word_back_2=" + prev_token2 + "\tPOS_back_2=" + prev_POS_tag2 + "\tBIO_back_2=" + prev_BIO_tag2 + "\tstemmed_word_back_2=" \
                        + stemmed_prev_token2 + "\t" 
    
    else:
        prev_info1 = "Beginning Sentence\t"

    next_info1 = ""
    next_info2 = ""
    if training_features[i+1]:
        next_t1 = training_features[i+1]
        next_token1 = next_t1[0]
        next_POS_tag1 = next_t1[1]
        next_BIO_tag1 = next_t1[2]
        stemmed_next_token1 = ps.stem(next_token1)
        next_role_label1 = ""
        next_class_of_predicate1 = ""
        if len(next_t1) > 5:
                next_role_label1 = next_t1[5]
                if next_role_label1 == "PRED":
                    next_class_of_predicate1 = next_t1[6]
        next_info1 = "word_plus_1=" + next_token1 + "\tPOS_plus_1=" + next_POS_tag1 + "\tBIO_plus_1=" + next_BIO_tag1 + "\ttemmed_word_plus_1=" \
                            + stemmed_next_token1 + "\t" 

        if training_features[i+2]:
            next_t2 = training_features[i+2]
            next_token2 = next_t2[0]
            next_POS_tag2 = next_t2[1]
            next_BIO_tag2 = next_t2[2]
            stemmed_next_token2 = ps.stem(next_token2)
            next_role_label2 = ""
            next_class_of_predicate2 = ""
            if len(next_t2) > 5:
                    next_role_label2 = next_t2[5]
                    if next_role_label2 == "PRED":
                        next_class_of_predicate2 = next_t2[6]
            next_info2 = "word_plus_2=" + next_token2 + "\tPOS_plus_2=" + next_POS_tag2 + "\tBIO_plus_2=" + next_BIO_tag2 + "\ttemmed_word_plus_2=" \
                            + stemmed_next_token2 + "\t" 

    
    else:
        next_info1 = "Ending Sentence\t"


    if len(t) > 5:
            role_label = t[5]
            if role_label == "ARG1":
                arg1_index = int(t[3])

    sentence.append(token)


    line = line = "word=" + token + "\tPOS=" + POS_tag + "\tBIO=" + BIO_tag + "\tstemmed_word=" + stemmed_token + \
                "\t" + prev_info1 + prev_info2 + next_info1 + next_info2
    text_lines.append(line)
    # f.write(line)

f.close()

file = open(test_file,"r")
lines = file.readlines()
file.close()

f = open("test.feature", "w")
test_features = []
for line in lines:
    t = line.split()
    test_features.append(t)
    
for i in range(len(test_features)):
    t = test_features[i]
    if len(t) == 0:
        # Embeddings of N-grams
        text = ' '.join(sentence)
        # print(text)
        doc = nlp(text)
        length = len(sentence)
        pq_unigram = PriorityQueue()
        pq_forward_bigram = PriorityQueue()
        pq_backward_bigram = PriorityQueue()
        pq_forward_trigram = PriorityQueue()
        pq_backward_trigram = PriorityQueue()
        for j in range(length):
            # print(doc[j])
            # Vector for ARG1 unigrams
            unigram_embed = doc[j].vector
            # Vector for forward ARG1 bigrams
            forward_bigram = doc[j:j+2].vector
            # Vector for backward ARG1 bigrams
            backward_bigram = doc[j-1:j+1].vector
            # Vector for forward ARG1 trigrams
            forward_trigram = doc[j:j+3].vector
            # Vector for backward ARG1 trigrams
            backward_trigram = doc[j-2:j+1].vector

            # forward_bigram_similarity = 0
            # backward_bigram_similarity = 0
            # forward_trigram_similarity = 0
            # backward_trigram_similarity = 0

            unigram_embed_similarity = np.dot(global_unigram_embed, unigram_embed)/(norm(global_unigram_embed)*norm(unigram_embed))
            # if not (forward_bigram == np.zeros(300,)).all():
            forward_bigram_similarity = np.dot(global_forward_bigram, forward_bigram)/(norm(global_forward_bigram)*norm(forward_bigram))
            # if not (backward_bigram == np.zeros(300,)).all():
            backward_bigram_similarity = np.dot(global_backward_bigram, backward_bigram)/(norm(global_backward_bigram)*norm(backward_bigram))
            # if not (forward_trigram == np.zeros(300,)).all():
            forward_trigram_similarity = np.dot(global_forward_trigram, forward_trigram)/(norm(global_forward_trigram)*norm(forward_trigram))
            # if not (backward_trigram == np.zeros(300,)).all():
            backward_trigram_similarity = np.dot(global_backward_trigram, backward_trigram)/(norm(global_backward_trigram)*norm(backward_trigram))

            if not math.isnan(unigram_embed_similarity):
                pq_unigram.put(-unigram_embed_similarity)
            if not math.isnan(forward_bigram_similarity):
                pq_forward_bigram.put(-forward_bigram_similarity)
            if not math.isnan(backward_bigram_similarity):
                pq_backward_bigram.put(-backward_bigram_similarity)
            if not math.isnan(forward_trigram_similarity):
                pq_forward_trigram.put(-forward_trigram_similarity)
            if not math.isnan(backward_trigram_similarity):
                pq_backward_trigram.put(-backward_trigram_similarity)


            text_line = text_lines[j]
            text_line += "unigram_embed_similarity=" + str(unigram_embed_similarity) + "\tforward_bigram_similarity=" + str(forward_bigram_similarity) + \
                            "\tbackward_bigram_similarity=" + str(backward_bigram_similarity) + "\tforward_trigram_similarity=" \
                        + str(forward_trigram_similarity) + "\tbackward_trigram_similarity=" + str(backward_trigram_similarity) + "\n"
            
            # print(text_line)
            f.write(text_line)
        
        # print("ranking")
        top_unigram = ""
        top_forward_bigram = ""
        top_backward_bigram = ""
        top_forward_trigram = ""
        top_backward_trigram = ""
        # print(pq_unigram.qsize())
        # print(pq_forward_bigram.qsize())
        # print(pq_backward_bigram.qsize())
        # print(pq_forward_trigram.qsize())
        # print(pq_backward_trigram.qsize())

        for n in range(5):
            if not pq_unigram.empty():
                top_unigram += "top_" + str(n+1) + "_embed_unigram="+str((-1)*pq_unigram.get()) + "\t"
            if not pq_forward_bigram.empty():
                top_forward_bigram += "top_" + str(n+1) + "_embed_forward_bigram=" + str((-1)*pq_forward_bigram.get()) + "\t"
            if not pq_backward_bigram.empty():
                top_backward_bigram += "top_" + str(n+1) + "_embed_backward_bigram=" + str((-1)*pq_backward_bigram.get()) + "\t"
            if not pq_forward_trigram.empty():
                top_forward_trigram += "top_" + str(n+1) + "_embed_forward_trigram=" + str((-1)*pq_forward_trigram.get()) + "\t"
            if not pq_backward_trigram.empty():
                top_backward_trigram += "top_" + str(n+1) + "_embed_backward_trigram=" + str((-1)*pq_backward_trigram.get()) + "\t"

        top_unigram += "\n"
        top_forward_bigram += "\n"
        top_backward_bigram += "\n"
        top_forward_trigram += "\n"
        top_backward_trigram += "\n"
        f.write(top_unigram)
        f.write(top_forward_bigram)
        f.write(top_backward_bigram)
        f.write(top_forward_trigram)
        f.write(top_backward_trigram)




        line = "\n"
        f.write(line)
        sentence.clear()
        text_lines.clear()
        # print("end")
        continue
    token = t[0]
    POS_tag = t[1]
    BIO_tag = t[2]
    role_label = ""
    class_of_predicate = ""
    stemmed_token = ps.stem(token)
    if len(t) > 5:
        role_label = t[5]
        if role_label == "PRED":
            class_of_predicate = t[6]
    prev_info1 = ""
    prev_info2 = ""
    if test_features[i-1]:
        prev_t1 = test_features[i-1]
        prev_token1 = prev_t1[0]
        prev_POS_tag1 = prev_t1[1]
        prev_BIO_tag1 = prev_t1[2]
        stemmed_prev_token1 = ps.stem(prev_token1)
        prev_role_label1 = ""
        prev_class_of_predicate1 = ""
        if len(prev_t1) > 5:
            prev_role_label1 = prev_t1[5]
            if prev_role_label1 == "PRED":
                prev_class_of_predicate1 = prev_t1[6]

        prev_info1 = "word_back_1=" + prev_token1 + "\tPOS_back_1=" + prev_POS_tag1 + "\tBIO_back_1=" + prev_BIO_tag1 + "\tstemmed_word_back_1=" \
                        + stemmed_prev_token1 + "\t" 

        if test_features[i-2]:
            prev_t2 = test_features[i-2]
            prev_token2 = prev_t2[0]
            prev_POS_tag2 = prev_t2[1]
            prev_BIO_tag2 = prev_t2[2]
            stemmed_prev_token2 = ps.stem(prev_token2)
            prev_role_label2 = ""
            prev_class_of_predicate2 = ""
            if len(prev_t2) > 5:
                prev_role_label2 = prev_t2[5]
                if prev_role_label2 == "PRED":
                    prev_class_of_predicate2 = prev_t2[6]

            prev_info2 = "word_back_2=" + prev_token2 + "\tPOS_back_2=" + prev_POS_tag2 + "\tBIO_back_2=" + prev_BIO_tag2 + "\tstemmed_word_back_2=" \
                        + stemmed_prev_token2 + "\t" 
    
    else:
        prev_info1 = "Beginning Sentence\t"

    next_info1 = ""
    next_info2 = ""
    if test_features[i+1]:
        next_t1 = test_features[i+1]
        next_token1 = next_t1[0]
        next_POS_tag1 = next_t1[1]
        next_BIO_tag1 = next_t1[2]
        stemmed_next_token1 = ps.stem(next_token1)
        next_role_label1 = ""
        next_class_of_predicate1 = ""
        if len(next_t1) > 5:
                next_role_label1 = next_t1[5]
                if next_role_label1 == "PRED":
                    next_class_of_predicate1 = next_t1[6]
        next_info1 = "word_plus_1=" + next_token1 + "\tPOS_plus_1=" + next_POS_tag1 + "\tBIO_plus_1=" + next_BIO_tag1 + "\ttemmed_word_plus_1=" \
                            + stemmed_next_token1 + "\t" 

        if test_features[i+2]:
            next_t2 = test_features[i+2]
            next_token2 = next_t2[0]
            next_POS_tag2 = next_t2[1]
            next_BIO_tag2 = next_t2[2]
            stemmed_next_token2 = ps.stem(next_token2)
            next_role_label2 = ""
            next_class_of_predicate2 = ""
            if len(next_t2) > 5:
                    next_role_label2 = next_t2[5]
                    if next_role_label2 == "PRED":
                        next_class_of_predicate2 = next_t2[6]
            next_info2 = "word_plus_2=" + next_token2 + "\tPOS_plus_2=" + next_POS_tag2 + "\tBIO_plus_2=" + next_BIO_tag2 + "\ttemmed_word_plus_2=" \
                            + stemmed_next_token2 + "\t" 

    
    else:
        next_info1 = "Ending Sentence\t"


#     if len(t) > 5:
#             role_label = t[5]
#             if role_label == "ARG1":
#                 arg1_index = int(t[3])

    sentence.append(token)


    line = line = "word=" + token + "\tPOS=" + POS_tag + "\tBIO=" + BIO_tag + "\tstemmed_word=" + stemmed_token + \
                "\t" + prev_info1 + prev_info2 + next_info1 + next_info2
    text_lines.append(line)
    # f.write(line)

f.close()


/Users/yulinhu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in double_scalars
/Users/yulinhu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:143: RuntimeWarning: invalid value encountered in double_scalars
/Users/yulinhu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in double_scalars
/Users/yulinhu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:137: RuntimeWarning: invalid value encountered in double_scalars
/Users/yulinhu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:141: RuntimeWarning: invalid value encountered in double_scalars
/Users/yulinhu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:357: RuntimeWarning: invalid value encountered in double_scalars
/Users/yulinhu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:361: RuntimeWarning: invalid value encountered in double_

In [63]:
!javac -cp maxent-3.0.0.jar:trove.jar *.java

In [64]:
!java -cp .:maxent-3.0.0.jar:trove.jar MEtrain training.feature model.chunk

Indexing events using cutoff of 4

	Computing event counts...  done. 72001 events
	Indexing...  Dropped event :[top_1_embed_forward_bigram=0.6969973577680209, top_2_embed_forward_bigram=0.6896743666900461, top_3_embed_forward_bigram=0.6531594040835632, top_4_embed_forward_bigram=0.613919792710593, top_5_embed_forward_bigram=0.604252332693222]
Dropped event :[top_1_embed_backward_bigram=0.7084207189030877, top_2_embed_backward_bigram=0.6888325340146843, top_3_embed_backward_bigram=0.6174424061051381, top_4_embed_backward_bigram=0.5926097336264324, top_5_embed_backward_bigram=0.5786507730128951]
Dropped event :[top_1_embed_forward_trigram=0.747257751496334, top_2_embed_forward_trigram=0.6919650213497304, top_3_embed_forward_trigram=0.6864788160272216, top_4_embed_forward_trigram=0.6798231533630975, top_5_embed_forward_trigram=0.6016185001776364]
Dropped event :[top_1_embed_backward_trigram=0.753139191122452, top_2_embed_backward_trigram=0.6583652281074355, top_3_embed_backward_trigram=0.

Dropped event :[top_1_embed_backward_bigram=0.7518016338891558, top_2_embed_backward_bigram=0.6904944765835649, top_3_embed_backward_bigram=0.6409874118213098, top_4_embed_backward_bigram=0.640575978582498, top_5_embed_backward_bigram=0.6323184700297134]
Dropped event :[top_1_embed_backward_trigram=0.8115523556306338, top_2_embed_backward_trigram=0.7493022474475267, top_3_embed_backward_trigram=0.7263506961775845, top_4_embed_backward_trigram=0.6969623485225733, top_5_embed_backward_trigram=0.6345561580626904]
Dropped event :[top_1_embed_forward_trigram=0.6971657105776414, top_2_embed_forward_trigram=0.6919357623646138, top_3_embed_forward_trigram=0.6917537146312654, top_4_embed_forward_trigram=0.6861539406407516, top_5_embed_forward_trigram=0.6647297712883743]
Dropped event :[top_1_embed_forward_bigram=0.7428152468027113, top_2_embed_forward_bigram=0.7047515302974456, top_3_embed_forward_bigram=0.6795649585612313, top_4_embed_forward_bigram=0.6552838860109188, top_5_embed_forward_bigr

Dropped event :[top_1_embed_forward_trigram=0.832030961477949, top_2_embed_forward_trigram=0.8146329640789001, top_3_embed_forward_trigram=0.770393882536759, top_4_embed_forward_trigram=0.754499254404065, top_5_embed_forward_trigram=0.7509234709869179]
Dropped event :[top_1_embed_backward_trigram=0.7615743481915573, top_2_embed_backward_trigram=0.7607974763775924, top_3_embed_backward_trigram=0.7368152683707544, top_4_embed_backward_trigram=0.7116924591047942, top_5_embed_backward_trigram=0.7112361061409304]
Dropped event :[top_1_embed_forward_trigram=0.8284950155381228, top_2_embed_forward_trigram=0.7801435614370869, top_3_embed_forward_trigram=0.7721014979820362, top_4_embed_forward_trigram=0.7584523161226058, top_5_embed_forward_trigram=0.7427931563243845]
Dropped event :[top_1_embed_backward_trigram=0.7900530235226496, top_2_embed_backward_trigram=0.7524797873830036, top_3_embed_backward_trigram=0.732342178564261, top_4_embed_backward_trigram=0.7210171614054931, top_5_embed_backwar

Dropped event :[top_1_embed_forward_trigram=0.7650328035545929, top_2_embed_forward_trigram=0.74678468435569, top_3_embed_forward_trigram=0.7417229835049469, top_4_embed_forward_trigram=0.7333438799282685, top_5_embed_forward_trigram=0.7264034984953456]
Dropped event :[top_1_embed_forward_bigram=0.7273688777849862, top_2_embed_forward_bigram=0.7218131550032637, top_3_embed_forward_bigram=0.661224898090694, top_4_embed_forward_bigram=0.6463306376898437, top_5_embed_forward_bigram=0.6435494157921336]
Dropped event :[top_1_embed_backward_bigram=0.7462427219552035, top_2_embed_backward_bigram=0.7197636169046915, top_3_embed_backward_bigram=0.6640979912930793, top_4_embed_backward_bigram=0.6546442993343558, top_5_embed_backward_bigram=0.6280071228948435]
Dropped event :[top_1_embed_forward_trigram=0.7267629237369202, top_2_embed_forward_trigram=0.7124091109526423, top_3_embed_forward_trigram=0.7093327934535535, top_4_embed_forward_trigram=0.6503849833483953, top_5_embed_forward_trigram=0.63

Dropped event :[top_1_embed_backward_trigram=0.6272892555624761, top_2_embed_backward_trigram=0.5316086797728385, top_3_embed_backward_trigram=0.5275498993612537, top_4_embed_backward_trigram=0.4949228248411001, top_5_embed_backward_trigram=0.44894314749730463]
Dropped event :[top_1_embed_forward_trigram=0.7602774514294629, top_2_embed_forward_trigram=0.7268418960717951, top_3_embed_forward_trigram=0.7048487570994554, top_4_embed_forward_trigram=0.7027059892379592, top_5_embed_forward_trigram=0.6989180518534109]
Dropped event :[top_1_embed_forward_trigram=0.7602774514294629, top_2_embed_forward_trigram=0.7268418960717951, top_3_embed_forward_trigram=0.7048487570994554, top_4_embed_forward_trigram=0.7027059892379592, top_5_embed_forward_trigram=0.6989180518534109]
Dropped event :[top_1_embed_forward_trigram=0.7602774514294629, top_2_embed_forward_trigram=0.7268418960717951, top_3_embed_forward_trigram=0.7048487570994554, top_4_embed_forward_trigram=0.7027059892379592, top_5_embed_forwar

Sorting and merging events... done. Reduced 68868 events to 53298.
Done indexing.
Incorporating indexed data for training...  
done.
	Number of Event Tokens: 53298
	    Number of Outcomes: 3
	  Number of Predicates: 26885
...done.
Computing model parameters...
Performing 100 iterations.
  1:  .. loglikelihood=-75659.2310959844	0.8560870070279375
  2:  .. loglikelihood=-36692.94939272952	0.9684323633617936
  3:  .. loglikelihood=-29730.504900567536	0.9697827728407968
  4:  .. loglikelihood=-25566.237386514607	0.9731660568043213
  5:  .. loglikelihood=-22631.11479772013	0.97537317767323
  6:  .. loglikelihood=-20431.043827752786	0.9772027647092989
  7:  .. loglikelihood=-18707.599321453657	0.9788581053609804
  8:  .. loglikelihood=-17312.154740575457	0.9799471452634024
  9:  .. loglikelihood=-16153.584683330891	0.9809926235697276
 10:  .. loglikelihood=-15172.722380993642	0.9818202938955684
 11:  .. loglikelihood=-14329.257526949423	0.9823285125166986
 12:  .. loglikelihood=-13594.625418

In [65]:
!java -cp .:maxent-3.0.0.jar:trove.jar MEtag test.feature model.chunk response.chunk

In [73]:
input_file = "response.chunk"
output_file = "output1.txt"
file = open(input_file,"r")
lines = file.readlines()
file.close()

input_features = []
for line in lines:
    t = line.split()
    if len(t) == 0:
        input_features.pop()
        input_features.pop()
        input_features.pop()
        input_features.pop()
        input_features.pop()
    input_features.append(t)
# print(input_features)

f = open(output_file, "w")
for i in range(len(input_features)):
    t = input_features[i]
    # print(t)
    if len(t) == 0:
        line = "\n"
        f.write(line)
        continue
    token = t[0]
    arg1 = t[1]
#     if len(t) > 1:
#         arg1 = t[1]
    line = token[5:]
    if arg1 == "True":
        line += "\tARG1\n"
    else:
        line += "\n"

    f.write(line)



In [74]:
from arg_scorer import score_file_with_NNP_adjustment

In [75]:
score_file_with_NNP_adjustment('%_nombank.clean.test', 'output1.txt', 'ARG1')

System [40, 62, 103, 116, 154, 195, 239, 320, 355, 380, 397, 421, 445, 469, 501, 523, 660, 722, 756, 784, 834, 870, 932, 956, 988, 1142, 1187, 1232, 1277, 1298, 1313, 1328, 1457, 1482, 1490, 1525, 1547, 1570, 1592, 1615, 1642, 1664, 1693, 1739, 1744, 1752, 1799, 1810, 1828, 1839, 1863, 1941, 2099, 2128, 2161, 2221, 2414, 2458, 2487, 2502, 2527, 2597, 2611, 2633, 2645, 2661, 2664, 2674, 2690, 2693, 2703, 2719, 2722, 2732, 2748, 2751, 2893, 2932, 2959, 2966, 2989, 3062, 3086, 3116, 3183, 3248, 3276, 3292, 3321, 3356, 3370, 3376, 3403, 3427, 3457, 3479, 3566, 3647, 3705, 3734, 3762, 3817, 3827, 3841, 3851, 3865, 3898, 3934, 4072, 4120, 4130, 4160, 4206, 4237, 4275, 4302, 4308, 4319, 4322, 4330, 4340]
Answers [9, 62, 78, 103, 116, 154, 169, 195, 229, 257, 288, 320, 355, 380, 421, 438, 462, 501, 523, 557, 581, 612, 637, 660, 686, 722, 742, 756, 784, 834, 870, 904, 909, 932, 956, 976, 1013, 1057, 1096, 1142, 1168, 1169, 1170, 1171, 1223, 1224, 1225, 1277, 1298, 1313, 1328, 1345, 1352, 1353, 